In [2]:
import pandas as pd 
import numpy as np

In [3]:
properties = pd.read_csv("properties.csv", sep=",") 

In [4]:
neighbourhoods = pd.read_csv("neighbourhood.csv", sep=",")
neighbourhoods["neighbourhood"] = neighbourhoods["neighbourhood"].str.lower().str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf8")

Removendo anúncios

In [5]:
properties = properties.loc[properties["url"].dropna().index]
properties = properties.loc[properties["header"].dropna().index]

In [6]:
#properties["price"].unique()

In [7]:
len(properties)

9972

In [8]:
properties = properties.drop_duplicates(subset=["id", "header", "address", "area", "price"], 
                        keep="first")

In [9]:
properties = properties.reset_index(drop=False) 

In [10]:
properties["area"] = properties["area"].replace("--","0").astype(int)

In [11]:
properties["rooms"] = properties["rooms"].str.replace("--","0").str.split(" ").str[0].astype(int)

In [12]:
properties["price"] = properties["price"].str.split(" ").str[1].str.replace(".", "").astype(int)


<ipython-input-12-e7f6d0900307>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  properties["price"] = properties["price"].str.split(" ").str[1].str.replace(".", "").astype(int)


In [13]:
properties["garages"] = properties["garages"].str.replace("--","0").str.split(" ").str[0].astype(int)

In [14]:
#properties["condo"] = properties["condo"].str.split(" ").str[1].replace(np.nan,"0").astype(float)
properties["condo"] = properties["condo"].str.split(" ").str[1].str.replace(".","").replace(np.nan,"0").astype(int)

<ipython-input-14-9ee6443d3e2c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  properties["condo"] = properties["condo"].str.split(" ").str[1].str.replace(".","").replace(np.nan,"0").astype(int)


In [15]:
properties["bathrooms"] = properties["bathrooms"].str.split(" ").str[0].replace("--","0").astype(int)

In [16]:
properties["url"] = properties["url"].astype(str)

In [17]:
properties["header"] = properties["header"].astype(str)

In [18]:
properties = pd.concat([properties, properties["amenities"].str.get_dummies(sep="|")], axis=1)

In [19]:
properties = properties.drop(columns=["amenities"])

In [20]:
properties["crawled_at"] = pd.to_datetime(properties["crawled_at"], format="%Y-%m-%d %H:%M:%S")

In [21]:
for idx,address in enumerate(properties["address"].str.lower().str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf8")):
    for index,row in neighbourhoods.iterrows():
        if address.find(row["neighbourhood"]) != -1:
            if row["check_name"]:
                properties.loc[idx,"neighbourhood"] = row["equivalent"]
            else:
                properties.loc[idx,"neighbourhood"] = row["neighbourhood"]
            break

In [22]:
properties.to_csv("properties_v2.csv", index=False)